In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/yolo_v7

/content/drive/MyDrive/yolo_v7


In [ ]:
# !git clone https://github.com/WongKinYiu/yolov7.git

In [ ]:
%cd yolov7

/content/drive/MyDrive/yolo_v7/yolov7


## PREPARING .TXT and YAML FILES FOR YOLO_V7 INPUT

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import cv2
import os
from tqdm import *
import shutil as sh
from sklearn.model_selection import KFold

In [ ]:
# Reading files

df = pd.read_csv('train/train_labels.csv')
df.LabelName.value_counts()

# # Finding H/W for an image

# df['image_width'] = 0
# df['image_height'] = 0

# for img_name in df.ImageID.unique():
#     img_shape = cv2.imread(f'train/images/{img_name}').shape
#     h = img_shape[0]
#     w = img_shape[1]

#     df.loc[df.ImageID == img_name, 'image_width'] = w
#     df.loc[df.ImageID == img_name, 'image_height'] = h

pothole    1371
Name: LabelName, dtype: int64

In [ ]:
# Bounding Box Attributes
df['w'] = df.XMax - df.XMin
df['h'] = df.YMax - df.YMin
df['area'] = df.w * df.h

# Normalizing
df.w = df.w / df.image_width
df.h = df.h / df.image_height

df['x_center'] = df.XMin/df.image_width + (df.w/2)
df['y_center'] = df.YMin/df.image_height + (df.h/2)

le=LabelEncoder()
df['classes'] = le.fit_transform(df.LabelName)

df['path']=df.ImageID.apply(lambda x: "train/images/"+x)

In [ ]:
def prep_path(train_df,valid_df,fold):
  for name,mini in tqdm(train_df.groupby('ImageID')):
      path2save = f'train_{fold}_fold/'

      if not os.path.exists('train_data/labels/'+path2save):
          os.makedirs('train_data/labels/'+path2save)

      with open('train_data/labels/'+path2save+name.replace('.jpg','')+".txt", 'w+') as f:
          row = mini[['classes','x_center','y_center','w','h']].astype(float).values
          row = row.astype(str)
          for j in range(len(row)):
              text = ' '.join(row[j])
              f.write(text)
              f.write("\n") 
      if not os.path.exists('train_data/images/{}'.format(path2save)):
          os.makedirs('train_data/images/{}'.format(path2save))
      sh.copy(mini.path.values[0],'train_data/images/{}{}'.format(path2save,name))


  for name,mini in tqdm(valid_df.groupby('ImageID')):
      path2save = f'val_{fold}_fold/'

      if not os.path.exists('train_data/labels/'+path2save):
                  os.makedirs('train_data/labels/'+path2save)

      with open('train_data/labels/'+path2save+name.replace('.jpg','')+".txt", 'w+') as f:
          row = mini[['classes','x_center','y_center','w','h']].astype(float).values
          row = row.astype(str)
          for j in range(len(row)):
              text = ' '.join(row[j])
              f.write(text)
              f.write("\n") 
      if not os.path.exists('train_data/images/{}'.format(path2save)):
          os.makedirs('train_data/images/{}'.format(path2save))
      sh.copy(mini.path.values[0],'train_data/images/{}{}'.format(path2save,name))


  import yaml
  dic_data={"train": f"train_data/images/train_{fold}_fold/",
              "val": f"train_data/images/val_{fold}_fold/",
              "path": "../",
              "nc": 1,
              "names":['pothole'] }


  with open(f'pothole_yaml_{fold}.yaml', 'w') as outfile:
      yaml.dump(dic_data, outfile, default_flow_style=False)

## K-FOLD TECHNIQUE

In [ ]:
k = 5
kf = KFold(n_splits=k)
fold=1
image_paths_valid = pd.DataFrame(df.ImageID.unique())
for train_index, test_index in kf.split(image_paths_valid):
    print(fold)
    X_train, X_test = image_paths_valid.iloc[train_index], image_paths_valid.iloc[test_index]
    valid_ids = X_test[0].values
    train_df = df.loc[~df.ImageID.isin(valid_ids)]
    valid_df = df.loc[df.ImageID.isin(valid_ids)]
    train_df.shape,valid_df.shape
    prep_path(train_df,valid_df,fold)
    
    fold += 1

1


100%|██████████| 135/135 [00:01<00:00, 74.23it/s]


2


100%|██████████| 135/135 [00:02<00:00, 54.50it/s]


3


100%|██████████| 135/135 [00:01<00:00, 76.67it/s]


4


100%|██████████| 135/135 [00:01<00:00, 78.15it/s]


5


100%|██████████| 134/134 [00:02<00:00, 60.86it/s]
